In [1]:
pip install --upgrade qiskit qiskit-machine-learning qiskit-algorithms numpy matplotlib


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from qiskit_aer import Aer
from qiskit.primitives import StatevectorSampler  # ✅ Updated to latest sampler
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGatesDecomposition, CommutativeCancellation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# ✅ Step 1: Generate and Preprocess Data
X, y = make_classification(n_samples=100, 
                           n_features=2,    # Total number of features
                           n_informative=2, # Ensure sum ≤ total features
                           n_redundant=0,   # No redundant features
                           n_clusters_per_class=1, 
                           random_state=42)

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ✅ Step 2: Define Quantum Feature Map
feature_map = ZZFeatureMap(feature_dimension=2, reps=2, entanglement='linear')

# ✅ Step 3: Define Variational Quantum Circuit (Ansatz)
ansatz = RealAmplitudes(num_qubits=2, reps=3, entanglement='linear')

# ✅ Step 4: Initialize the StatevectorSampler (Updated for Qiskit 1.2+)
sampler = StatevectorSampler()

# ✅ Step 5: Define Fidelity Quantum Kernel
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=sampler)

# ✅ Step 6: Instantiate the Optimizer (Corrected Import)
optimizer = COBYLA(maxiter=100)  # ✅ Ensure optimizer is an instance, not a string

# ✅ Step 7: Define a Custom Pass Manager for Transpilation (Fixes Warnings)
pass_manager = PassManager([
    Optimize1qGatesDecomposition(basis=['u3', 'cx']),
    CommutativeCancellation()
])

# ✅ Step 8: Define the Variational Quantum Classifier (VQC)
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    sampler=sampler,
    pass_manager=pass_manager  # ✅ Provide pass manager to avoid transpilation warnings
)

# ✅ Step 9: Train the VQC Model
vqc.fit(X_train, y_train)

# ✅ Step 10: Evaluate the Model
y_pred = vqc.predict(X_test)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"VQC Accuracy: {accuracy * 100:.2f}%")


VQC Accuracy: 75.00%
